In [2]:
!pip install pandas numpy scikit-learn

In [5]:
import pandas as pd
import ast
from collections import Counter
import numpy as np
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_extraction.text import CountVectorizer
import re

def clean_stem_clean_column(text):
    """Membersihkan dan mengkonversi string list menjadi list yang valid"""
    if pd.isna(text) or text == '' or text == '[]':
        return []

    # Jika sudah berupa list, kembalikan langsung
    if isinstance(text, list):
        return text

    # Jika berupa string, coba parse
    try:
        # Hapus spasi berlebih dan karakter yang tidak perlu
        text = text.strip()
        if text.startswith('[') and text.endswith(']'):
            # Gunakan ast.literal_eval untuk parse list yang aman
            return ast.literal_eval(text)
        else:
            # Jika bukan format list, split berdasarkan koma
            return [word.strip().strip("'\"") for word in text.split(',') if word.strip()]
    except:
        # Jika gagal parse, return list kosong
        return []

def hitung_information_gain(X, y):
    """Menghitung Information Gain untuk setiap fitur"""
    # Gunakan mutual information sebagai proxy untuk Information Gain
    mi_scores = mutual_info_classif(X, y, random_state=42)
    return mi_scores

def hitung_frekuensi_dengan_seleksi_fitur():
    """Menghitung frekuensi kata dengan seleksi fitur menggunakan Information Gain"""

    # Baca file xlsx
    print("Membaca file data pengujian.xlsx...")
    df = pd.read_excel('data pengujian.xlsx')

    # Bersihkan data
    print("Membersihkan data...")
    df['stem_clean_processed'] = df['stem_clean'].apply(clean_stem_clean_column)

    # Konversi list kata menjadi string untuk CountVectorizer
    df['text_processed'] = df['stem_clean_processed'].apply(lambda x: ' '.join(x) if x else '')

    # Hapus baris yang kosong
    df = df[df['text_processed'].str.strip() != '']

    # Gunakan CountVectorizer untuk membuat bag of words
    print("Membuat bag of words...")
    vectorizer = CountVectorizer(
        lowercase=True,
        token_pattern=r'\b\w+\b',
        min_df=2,  # Minimal dokumen yang mengandung kata
        max_df=0.95  # Maksimal 95% dokumen
    )

    # Fit dan transform data
    X = vectorizer.fit_transform(df['text_processed'])
    feature_names = vectorizer.get_feature_names_out()

    # Konversi label ke numerik
    label_mapping = {'positif': 0, 'negatif': 1, 'netral': 2}
    y = df['sentiment_label'].map(label_mapping)

    # Hitung Information Gain
    print("Menghitung Information Gain...")
    information_gain = hitung_information_gain(X, y)

    # Buat DataFrame dengan hasil
    feature_info = pd.DataFrame({
        'Kata': feature_names,
        'Frekuensi': np.array(X.sum(axis=0)).flatten(),
        'Information_Gain': information_gain
    })

    # Urutkan berdasarkan Information Gain tertinggi
    feature_info = feature_info.sort_values('Information_Gain', ascending=False)

    # Reset index
    feature_info = feature_info.reset_index(drop=True)

    # Simpan ke file excel
    output_file = 'hasil_frekuensi_dengan_seleksi_fitur.xlsx'
    feature_info.to_excel(output_file, index=False, engine='openpyxl')


    print(f"Hasil disimpan ke: {output_file}")
    print(f"Total kata unik: {len(feature_info)}")
    print(f"Total frekuensi kata: {feature_info['Frekuensi'].sum()}")

    # Tampilkan 10 kata dengan Information Gain tertinggi
    print("\n10 Kata dengan Information Gain Tertinggi:")
    print(feature_info.head(10).to_string(index=False))

    # Tampilkan statistik Information Gain
    print(f"\nStatistik Information Gain:")
    print(f"Rata-rata: {feature_info['Information_Gain'].mean():.6f}")
    print(f"Minimum: {feature_info['Information_Gain'].min():.6f}")
    print(f"Maximum: {feature_info['Information_Gain'].max():.6f}")

    return feature_info

if __name__ == "__main__":
    print("=== SCRIPT DENGAN SELEKSI FITUR ===")
    print("Menghitung frekuensi kata dengan seleksi fitur menggunakan Information Gain")
    print("=" * 70)

    try:
        hasil = hitung_frekuensi_dengan_seleksi_fitur()
        print("\nScript berhasil dijalankan!")
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()

=== SCRIPT DENGAN SELEKSI FITUR ===
Menghitung frekuensi kata dengan seleksi fitur menggunakan Information Gain
Membaca file data pengujian.csv...
Membersihkan data...
Membuat bag of words...
Menghitung Information Gain...
Hasil disimpan ke: hasil_frekuensi_dengan_seleksi_fitur.xlsx
Total kata unik: 218
Total frekuensi kata: 1338

10 Kata dengan Information Gain Tertinggi:
    Kata  Frekuensi  Information_Gain
   tiket         42          0.051422
   bayar         22          0.039320
   harga         26          0.037017
     gak         19          0.034387
  kecewa         13          0.032645
aplikasi         17          0.032462
 pesawat          9          0.031945
   hotel         32          0.031316
  refund         11          0.026621
maskapai          9          0.026223

Statistik Information Gain:
Rata-rata: 0.009003
Minimum: 0.000771
Maximum: 0.051422

Script berhasil dijalankan!
